<img src="http://193.175.187.164/static/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="100" width="100"  align="left"/>
<img src="http://reiner-lemoine-institut.de//wp-content/uploads/2015/09/rlilogo.png" alt="RLI" height="100" width="100" align="right"/>

# OpenEnergyPlatform
<br>
## Usage of OpenEnergyPlatform API-Dialect (oedialect)
Repository: https://github.com/openego/oedialect

In [ ]:
__copyright__ = "Reiner Lemoine Institut"
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://github.com/openego/data_processing/blob/master/LICENSE"
__author__    = "christian-rli"

In [ ]:
import pandas as pd
import getpass
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
import oedialect

## Reading data from a csv file and uploading it to the oedb

Pandas has a read_csv function which makes importing a csv-file rather comfortable. It reads csv into a DataFrame. By default, it assumes that the fields are comma-separated. 

Our example file has columns with semicolons as separators, so we have to specify this when reading the file. Make sure to adapt the path to the file you're using.


In [ ]:
example_df = pd.read_csv('/home/local/RL-INSTITUT/christian.hofmann/data/SzenDB_DataTemplate.csv', encoding='utf8', sep=';')

Looking at the first three lines of our dataframe:

In [ ]:
example_df[:1]

## Connection to OEP

If we want to upload data to the OEP in we first need to connect to it, using our OEP user name and token. 
Note: You ca view your token in your OEP profile site after loggin in.  


In [ ]:
# White spaces in the username are fine!
user = input('Enter OEP-username:')
token = getpass.getpass('Token:')

Now we'll create an sql-alchemy-engine. The engine is what 'speaks' oedialect to the data base api. We need to tell it where the data base is and pass our credentials.


In [ ]:
# Create Engine:
OEP_URL = 'oep.iks.cs.ovgu.de'
OED_STRING = f'postgresql+oedialect://{user}:{token}@{OEP_URL}'

engine = sa.create_engine(OED_STRING)
metadata = sa.MetaData(bind=engine)

## Setup a Table

We need to tell the data base what columns and datatypes we are about to upload. In our case we have four columns, two of which are text, one is integer and the last is float.

In [ ]:
table_name = 'example_dialect_table'
schema_name = 'sandbox'

ExampleTable = sa.Table(
    table_name,
    metadata,
    sa.Column('variable', sa.VARCHAR(50)),
    sa.Column('unit', sa.VARCHAR(50)),
    sa.Column('year', sa.INTEGER),
    sa.Column('value', sa.FLOAT(50)),
    schema=schema_name
)

## Create the new Table

Now we tell our engine to connect to the data base and create the defined table within the chosen schema.

In [ ]:
conn = engine.connect()
print('Connection established')
if not engine.dialect.has_table(conn, table_name, schema_name):
    ExampleTable.create()
    print('Created table')
else:
    print('Table already exists')

 ## Insert data into Table
 
 Uploading the information from our DataFrame is now done with a simple command. (Connecting and uploading may take a minute.)

In [ ]:
example_df.to_sql(table_name, conn, schema='sandbox', if_exists='append')

You can also insert data manually into the table.

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
try:
    insert_statement = ExampleTable.insert().values(
        [
            dict(variable='fairy dust', unit='t', year=2020, value=21),
            dict(variable='mana', unit='kg', year=1999, value=21),
            dict(variable='the force', unit='l', year=1998, value=21)
        ]
    )
    session.execute(insert_statement)
    session.commit()
    print('Insert successful!')
except Exception as e:
    session.rollback()
    raise
    print('Insert incomplete!')
finally:
    session.close()

## Select from Table

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
print(session.query(ExampleTable).all())
session.close()

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
df = pd.DataFrame(session.query(ExampleTable).all())
session.close()
df

## Update data in Table

In [ ]:
example_df.dtypes

In [ ]:
example_df.columns